In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Conv2D
from keras.layers import MaxPool2D,Flatten,Dropout,ZeroPadding2D

In [ ]:
alphabet = pd.read_csv('../input/handwritten_data_785.csv')
#shuffle the data set
alphabet=alphabet.sample(frac=1)
#split features and labels
images=alphabet.iloc[:,1:].values
raw_labels=alphabet.iloc[:,0].values.ravel()
print(images.shape,raw_labels.shape)

In [ ]:
#Just for test
print(raw_labels[0])
tmp=images[0].reshape((28,28))
plt.imshow(tmp)
plt.show()

In [ ]:
#do one hot encoding for labels
#aka if original label is 3
#after one-hot, it becomes a 26-long array
#[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]  
def one_hot(labels):
    num_labels=labels.shape[0]
    result=np.zeros((num_labels,26))
    offset=np.arange(num_labels)*26
    result.flat[offset+labels.ravel()]=1
    return result  
labels=one_hot(raw_labels)
print(labels[51211],raw_labels[51211])

In [ ]:
images=images.reshape(images.shape[0],28,28,1).astype("float32")
images=images/255

In [ ]:
#print(images[0].shape)  ---->  (28,28,1)
#plt.imshow(images[0][:,:,0])
print(images.shape,labels.shape)

In [ ]:
#X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.3)

In [ ]:
#print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

In [ ]:
# Built the model
model = Sequential()

In [ ]:
#Layer-1
#Basic Convolutional layer===========
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),input_shape=(28,28,1)))
#ReLU Layer=======
#An activation layer, we do it after every Convolutional layer.
#How it works?  If input is x, then output is  max(0, x)
model.add(Activation('relu'))
#Normalization Layer=========
#I dont quite understand this layer, anyway, it's doing normalization
#model.add(BatchNormalization(epsilon=1e-6,axis=1))
#Pooling Layer==========
#Reduce number of parameters and prevent OVERFITTING,usually the pool size is (2,2)
#MaxPool means, if we have a 2*2 block, we choose the biggest number 
#  4  5
#  8  1
#Then after MaxPool, the output is 8, a 1*1 block.
model.add(MaxPool2D(pool_size=(2,2)))

In [ ]:
#Layer-2
#Padding Layer===========
#Since each time we use Convolutional layer, the input size would become smaller,
#so we add a Padding, here we set it to (1,1),
#which means if input size is H*W, after this layer, it would become (H+1)*(W+1)
model.add(ZeroPadding2D((1,1)))

#Basic Convolutional layer===========
model.add(Conv2D(48,kernel_size=(3,3)))
#ReLU Layer=======
model.add(Activation('relu'))
#Normalization Layer=========
#model.add(BatchNormalization(epsilon=1e-6,axis=1))
#Pooling Layer==========
model.add(MaxPool2D(pool_size=(2,2)))

In [ ]:
#Layer-3
#Padding Layer===========
model.add(ZeroPadding2D((1,1)))
#Basic Convolutional layer===========
model.add(Conv2D(64,kernel_size=(2,2)))
#ReLU Layer=======
model.add(Activation('relu'))
#Normalization Layer=========
#model.add(BatchNormalization(epsilon=1e-6,axis=1))
#Pooling Layer==========
model.add(MaxPool2D(pool_size=(2,2)))

In [ ]:
#Fully Connected Layer --- aka Last Layer
#Dropout Layer===========
#This should always be in the last layer
#It randomly drops out some parameter, still it prevents OVERFITTING.
model.add(Dropout(0.25))
model.add(Flatten())

#Dense Layer==========
#Its job is to do classification
model.add(Dense(4096))  #3168
model.add(Activation('relu'))

model.add(Dense(26))
model.add(Activation('softmax'))

In [ ]:
#Compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train and Predict
#Epoch means the times you want to train,here I just set epoch=1, since one round takes about 30 mins.
#validation_split, I split 80% as training data, 20% as test data
#verbose=1 means print the log, =0 dont print
model.fit(images,labels,batch_size=64,epochs=1,verbose=1,validation_split=0.2)

In [ ]:
model.predict(np.array([images[0]]))

In [ ]:
model.save("178.h5")

In [ ]:
from keras.models import load_model
model11=load_model("178.h5")

In [ ]:
model11.predict(np.array([images[0]]))